# Overview
* bolt-lmm의 경우 5000명 이상의 샘플을 분석할 시 사용 해라
* 5000명보다 적을시 GCTA나 GEMMA를 이용해라
* case와 control이 balanced되어 있어야지 유용(unbalanced되어 있으면 SAIGE이용)


##  해당하는 프로그램을 진행하는 코드(minimal)
* run_example.sh - bolt-lmm
* ren_example_reml2.sh - bolt-reml 
<기본형태>
./bolt --bfile=geno --phenoFile=pheno.txt --phenoCol=phenoName
--lmm --LDscoresFile=tables/LDSCORE.1000G_EUR.tab.gz
--statsFile=stats.tab

<기본형태>
./bolt --bfile=geno --phenoFile=pheno.txt --phenoCol=phenoName
--reml --modelSnps=modelSnps.txt


In [ ]:
-bfile=EUR_subset \  # input data는 plink를 통해 나온 bed/bim/fam file 이용
    # 여러개의 bim file이 존재할 시 (염색체 별로) --bim=data.chr{1:22}.bim 이용
<임의 사용자 지정 설정>
    --remove=EUR_subset.remove \ #사용자 정의로 제외하거나 제거 할 리스트를 넣을 수 있음
    --exclude=EUR_subset.exclude \
    --exclude=EUR_subset.exclude2 \

<input data설정>
    --phenoFile=EUR_subset.pheno2.covars \ # phenotype이 따로 파일로서 존재할 시
    # first two column은 FID, IID이여야 한다.
    # fam 파일 가장 마지막 컬럼 일시에는 --phenoUseFam 을 사용하여 시행 (numeric 1,0,-9)
    --phenoCol=PHENO \ #위의 pheno file중 phenotype이 정의 되어 있는 column을 의미
    --covarFile=EUR_subset.pheno2.covars \ #pc들을 추출하ll
    여 분석하는 것이 매우 효과적!!!!!!!!!!!!!!!!!
    --covarCol=CAT_COV \ # categorical covariates -> 10개이상이면 불가
    --qCovarCol=QCOV{1:2} \ #quantitative covariates -> 모두 숫자여야 함

<association 분석>
    # 두가지의 통계적인 방법이 존재 함(bolt-lmm(non-infinitesimal) -inf(infinitesimal))
    --lmm \ # perform default(BOLT-LMM)
    --LDscoresFile=../tables/LDSCORE.1000G_EUR.tab.gz \
    # non-European일시 LDSC를 이용하여 계산을 시행해야 한다. (PLINK에서 matching을 시행해줌)
    --modelSnps=EUR_subset.modelSnps # random effect설정ㅎ줄시
    --numThreads=2 

#QC 에서 missing rate가 0.1이상인것은 무시 / missing data 도 무시
# hardy에 의해서 필터링을 하지 않기에 plink --hardy를 이용하여 확인을 해봐라~

-----------------------------------------------------------여기서 부터는 부수적인 분석----------
    --statsFile=example.stats \
    --dosageFile=EUR_subset.dosage.chr17first100 \
    --dosageFile=EUR_subset.dosage.chr22last100.gz \
    --dosageFidIidFile=EUR_subset.dosage.indivs \
    --statsFileDosageSnps=example.dosageSnps.stats \
    --impute2FileList=EUR_subset.impute2FileList.txt \
    --impute2FidIidFile=EUR_subset.impute2.indivs \
    --statsFileImpute2Snps=example.impute2Snps.stats \
    --dosage2FileList=EUR_subset.dosage2FileList.txt \
    --statsFileDosage2Snps=example.dosage2Snps.stats \
    2>&1 | tee example.log # log output written to stdout and stderr

# basic args:
# --bfile: prefix of PLINK genotype files (bed/bim/fam)
# --remove: list of individuals to remove (FID IID)
# --exclude: list of SNPs to exclude (rs###)
# --phenoFile: phenotype file
# --phenoCol: column of phenotype file containing phenotypes
# --covarFile: covariate file
# --covarCol: column(s) containing categorical covariate (multiple ok)
# --qCovarCol: column(s) containing quantitative covariates (array format)
# --modelSnps: subset of SNPs to use in GRMs
# --lmm: flag to perform default BOLT-LMM mixed model association
# --LDscoresFile: reference LD Scores (data table in separate download)
# --numThreads: multi-threaded execution


In [ ]:
# reml을 사용하여 heritability 와 gene correlation을 계산 할 수 있음
# variance components란 random variable 분석을 의미

../bolt \
    --bfile=EUR_subset \
    --phenoFile=EUR_subset.pheno2.covars \
    --exclude=EUR_subset.exclude2 \
    --phenoCol=PHENO \
    --phenoCol=QCOV1 \
    --modelSnps=EUR_subset.modelSnps2 \
    --reml \
    --numThreads=2 \
    2>&1 | tee example_reml2.log # log output written to stdout and stderr

### NOTE: This example just demonstrates parameter usage.
###       The algorithm is not robust with so few samples; we recommend N>5000.

# --bfile: prefix of PLINK genotype files (bed/bim/fam)
# --phenoFile: phenotype file
# --phenoCol: column(s) of phenotype file containing phenotypes
# --modelSnps: subset of SNPs to use in GRMs
# --reml: flag to perform default BOLT-REML variance components analysis
# --numThreads: multi-threaded execution

## BOLT-LMM association test statistics output
* statsFile에 존재함
 SNP: rs number or ID string #snp
 CHR: 유전자
 BP: physical (base pair) position
 GENPOS: genetic position either from bim file or interpolated from genetic map
 ALLELE1: first allele in bim file (usually the minor allele), used as the effect allele
16
 ALLELE0: second allele in bim file, used as the reference allele
 A1FREQ: frequency of first allele
 F_MISS: fraction of individuals with missing genotype at this SNP
 BETA: effect size from BOLT-LMM approximation to infinitesimal mixed model (effect size)
 SE: standard error of effect size  
 P_BOLT_LMM_INF: infinitesimal mixed model association test p-value
 P_BOLT_LMM: non-infinitesimal mixed model association test p-value

### bolt-REML
* terminal 마지막 부분에 나타남
* 모두 두가지의 경우 proceeds 되어져서 발생함, output을 저장할 시 아래와 같은 명령어를 사용함
./bolt [... list of options ...] 2>&1 | tee output.log

# UK biobank 분석 예제

* 8+thread를 사용하여 분석을 진행
* PCA를 이용하여 cost를 줄일 수 있음 (speed up!)
* using plink2 --pca approx or EIGENSOFT v6.0+ fastmode
* 6번쨰 column에 numeric 형태의 fam file이 필요함

In [1]:
./bolt \
--bed=ukb_cal_chr{1:22}_v2.bed \
--bim=ukb_snp_chr{1:22}_v2.bim \
--fam=ukb1404_cal_chr1_v2_CURRENT.fixCol6.fam \
--remove=bolt.in_plink_but_not_imputed.FID_IID.976.txt \
--remove=sampleQC/remove.nonWhite.FID_IID.txt \
--exclude=snpQC/autosome_maf_lt_0.001.txt \
--exclude=snpQC/autosome_missing_gt_0.1.txt \
--phenoFile=ukb4777.phenotypes.tab \
--phenoCol=height \
--covarFile=ukb4777.covars.tab.gz \
--covarCol=cov_ASSESS_CENTER \
--covarCol=cov_GENO_ARRAY \
--covarMaxLevels=30 \
--qCovarCol=cov_AGE \
--qCovarCol=cov_AGE_SQ \
--qCovarCol=PC{1:20} \
--LDscoresFile=tables/LDSCORE.1000G_EUR.tab.gz \
--geneticMapFile=tables/genetic_map_hg19.txt.gz \
--lmmForceNonInf \
--numThreads=8 \
--statsFile=bolt_460K_selfRepWhite.height.stats.gz \
--bgenFile=ukb_imp_chr{1:22}_v2.bgen \
--bgenMinMAF=1e-3 \
--bgenMinINFO=0.3 \
--sampleFile=ukb1404_imp_chr1_v2_s487406.sample \
--statsFileBgenSnps=bolt_460K_selfRepWhite.height.bgen.stats.gz --verboseStats

SyntaxError: invalid syntax (<ipython-input-1-ce533238a2fe>, line 1)

## case control 분석 시 중요한 점
* 3가지의 주요한 parameter이 존재함
< sample size, minor allele frequency, and case-control ratio >

* case의 비율이 최소한 10%이며 MAF > 0.1% 일시 효과적임 -> case가 더 적을 시에는 SAIGE이용해야 한다.